In [10]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os
from pathlib import Path
import re
import pickle
# ver qtos o modelo apenas de ner acertaria
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize 
import torch
from transformers import BertTokenizer,BertForTokenClassification
import numpy as np
import json   
from importlib import reload  # Python 3.4+
import random
import model as mod
from model import BertForChunkClassification
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

from torch.utils.data import DataLoader
from importlib import reload 
#from eval import predict
import eval
#import importlib
#importlib.reload(module)
import dataset
from dataset import InputFeatures, load_and_cache_examples
import functionsAval as f
from sklearn_crfsuite.metrics import flat_f1_score, flat_classification_report
import joblib
import os


In [11]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
def has_numbers(inputString):
    return str(bool(re.search(r'\d', inputString)))

print(has_numbers('metionina 5mg'))

def isstopword(p):
  return str(p in stopwords.words('portuguese'))

print(isstopword('pelo'))


True
True


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lisat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
def powerset(entidade):    
    lista=list()
    for i in range(1, len(entidade)+1):
        #lista.append(entidade[i-1])
        for j in range(len(entidade) - i + 1):
            lista.append(entidade[j:j + i])
    return lista
        
#print(list(powerset([4, 5, 6])))
print(powerset(['dor', 'no', 'peito']))


dicPostagger = f.load_obj('dic_postagger')
def tipoPostaggerTokens(token, dicPostagger):
    postagger = 'N' # na duvida é N
    if token.lower() in dicPostagger.keys():
        postagger = dicPostagger.get(token.lower())
    return postagger
tipoPostaggerTokens('coração', dicPostagger)

[['dor'], ['no'], ['peito'], ['dor', 'no'], ['no', 'peito'], ['dor', 'no', 'peito']]


'N'

In [13]:
f = reload(f)
reload(dataset)
reload(eval)
reload(mod)

# em numero de frases
BATCH=506
#BATCH=5
#BATCH=800
#BATCH=8000 
print('BATCH:', BATCH)

BATCH: 506


In [5]:
dicSentences_new_test = f.loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
print(dicSentences_new_test[0])
#print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))

sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        tokens = [tok[0] for tok in tokens]
        sentences.append(' '.join(tokens).strip())
#print(sentences[0])

tags, tokens = f.predictBERTNER_IO(sentences, 'all')
dic_predictions = f.getDicPredictions(tags, tokens)
dic_predictions_ner_bert = f.getDicPredictions(tags, tokens)
print(dic_predictions[0])
print('len(dic_predictions):', len(dic_predictions))
#print(dic_predictions[9])
f.save_obj('dic_predictions_results_ner_'+str(BATCH), dic_predictions)
#dic_predictions = f.load_obj('dic_predictions_results_ner_'+str(BATCH))

print('len(dicSentences_new_test):', len(dicSentences_new_test))
print('len(dic_predictions):', len(dic_predictions))

region_true_list, region_pred_list, lista_erros = f.getListaRegionsTruePred(BATCH, dicSentences_new_test, dic_predictions)
f.save_obj('region_true_list'+str(BATCH), region_true_list)
print('region_pred_list[:4]:', region_pred_list[:4])
print('region_true_list[:4]:', region_true_list[:4])
print('lista_erros[:8]:', lista_erros[:8])
print('len(lista_erros):', len(lista_erros))
print('len(set(lista_erros)):', len(set(lista_erros)))
print('len(region_true_list):', len(region_true_list))
print('len(region_pred_list):', len(region_pred_list))

print('-----Avaliando nested só com modelo de NER com BERT:-----')

print(classification_report(region_true_list, region_pred_list, digits=6))
print(confusion_matrix(region_true_list, region_pred_list))


Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
506
[[['Lucas', 0, 43], [',', 1, 48], ['74', 2, 50], ['anos', 3, 53], ['.', 4, 57]], []]
numero de sentencas no total: 506
idx2tag: {0: 'Teste', 1: 'Anatomia', 2: 'O', 3: 'Problema', 4: 'Tratamento', 5: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 506
len(dicSentences_new_test): 506
len(dic_predictions): 506
region_pred_list[:4]: ['Problema', 'Tratamento', 'Problema', 'Problema']
region_true_list[:4]: ['Problema', 'Tratamento', 'Problema', 'Problema']
lista_erros[:8]: [7, 8, 13, 13, 14, 15, 15, 15]
len(lista_erros): 383
len(set(lista_erros)): 169
len(region_true_list): 1140
len(region_pred_list): 1140
-----Avaliando nested só com modelo de NER com BERT:-----
              precision    recall  f1-score   support

    Anatomia   0.829268  0.346939  0.489209       196
           O   0.000000  0.000000  0.000000       149
    Problema   0.798319  0.843195  0.820144       338
      

In [6]:
tags[0]

['O', 'O', 'O', 'O', 'O']

In [7]:
tokens[0]

['Lucas', ',', '74', 'anos', '.']

In [37]:
dic_predictions_ner_bert[42]

[[['Paciente', 0],
  ['em', 1],
  ['tratamento', 2],
  ['para', 3],
  ['ICC', 4],
  ['diastólica', 5],
  ['há', 6],
  ['cerca', 7],
  ['de', 8],
  ['5', 9],
  ['a', 10],
  ['com', 11],
  ['melhora', 12],
  ['importante', 13],
  ['dos', 14],
  ['sintomas', 15],
  ['após', 16],
  ['início', 17],
  ['do', 18],
  ['tratamento', 19],
  ['.', 20]],
 [['ICC diastólica', [4, 5], 'Problema'], ['tratamento', [19], 'Tratamento']]]

In [35]:
dic_predictions_ner_bert[15]

[[['calcificação', 0],
  ['mitral', 1],
  ['e', 2],
  ['aórtica', 3],
  ['com', 4],
  ['refluxo', 5],
  ['leve', 6],
  ['.', 7]],
 [['calcificação', [0], 'Problema'],
  ['mitral', [1], 'Tratamento'],
  ['e', [2], 'Problema'],
  ['aórtica', [3], 'Anatomia'],
  ['refluxo leve', [5, 6], 'Problema']]]

In [27]:
dic_predictions

{0: [[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []],
 1: [[['Em', 0],
   ['acompanhamento', 1],
   ['no', 2],
   ['ambualtorio', 3],
   ['há', 4],
   ['5', 5],
   ['anos', 6],
   ['por', 7],
   ['FA', 8],
   [',', 9],
   ['uso', 10],
   ['de', 11],
   ['marevan', 12],
   ['5mg', 13],
   ['1', 14],
   ['x', 15],
   ['ao', 16],
   ['dia', 17],
   ['.', 18]],
  []],
 2: [[['Comorbidades', 0],
   [':', 1],
   ['DM', 2],
   ['há', 3],
   ['10', 4],
   ['anos', 5],
   ['em', 6],
   ['uso', 7],
   ['de', 8],
   ['metformina', 9],
   ['850mg', 10],
   ['3', 11],
   ['cp', 12],
   ['/', 13],
   ['dia', 14],
   [',', 15],
   ['acarbose', 16],
   ['1', 17],
   ['cp', 18],
   ['/', 19],
   ['dia', 20],
   ['e', 21],
   ['glicazida', 22],
   ['60mg', 23],
   ['2', 24],
   ['cp', 25],
   ['/', 26],
   ['dia', 27],
   ['e', 28],
   ['insulina', 29],
   ['(', 30],
   ['24', 31],
   ['-', 32],
   ['0', 33],
   ['-', 34],
   ['24', 35],
   [')', 36],
   ['.', 37]],
  []],
 3: [[['HAS', 

In [40]:
def somaPredictions(dic_predictions, dic_predictions_ner_bert):
    retorno = {}
    for key, value in dic_predictions_ner_bert.items():
        #print(value)
        valor = value.copy()
        value2=dic_predictions[key]
        retorno[key]=[valor[0], valor[1]+value2[1]]
        #if key>4:
        #    break
    #print(retorno)
    return retorno

r = somaPredictions(dic_predictions, dic_predictions_ner_bert)
r[15]

[[['calcificação', 0],
  ['mitral', 1],
  ['e', 2],
  ['aórtica', 3],
  ['com', 4],
  ['refluxo', 5],
  ['leve', 6],
  ['.', 7]],
 [['calcificação', [0], 'Problema'],
  ['mitral', [1], 'Tratamento'],
  ['e', [2], 'Problema'],
  ['aórtica', [3], 'Anatomia'],
  ['refluxo leve', [5, 6], 'Problema'],
  ['mitral', [1], 'Anatomia'],
  ['aórtica', [3], 'Anatomia']]]

In [14]:
pathModel2=r'C:\Users\lisat\OneDrive\jupyter notebook\Two-level-ner-for-nested-entity\segundo_nivel\model\model'

dicSentences_new_test = f.loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
print(dicSentences_new_test[0])
#print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))

sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        tokens = [tok[0] for tok in tokens]
        sentences.append(' '.join(tokens).strip())
#print(sentences[0])

tags, tokens = f.predictBERTNER_IO2(sentences, pathModel2)
dic_predictions = f.getDicPredictions(tags, tokens)
dic_predictions_ner_bert2 = f.getDicPredictions(tags, tokens)
print(dic_predictions[0])
print('len(dic_predictions):', len(dic_predictions))
#print(dic_predictions[9])
f.save_obj('dic_predictions_results_ner2_'+str(BATCH), dic_predictions)
#dic_predictions = f.load_obj('dic_predictions_results_ner_'+str(BATCH))

print('len(dicSentences_new_test):', len(dicSentences_new_test))
print('len(dic_predictions):', len(dic_predictions))

dic_predictions = somaPredictions(dic_predictions, dic_predictions_ner_bert)

region_true_list, region_pred_list, lista_erros = f.getListaRegionsTruePred(BATCH, dicSentences_new_test, dic_predictions)
#region_true_list, region_pred_list, lista_erros = f.getListaRegionsTruePred(BATCH, dicSentences_new_test, dic_predictions)

f.save_obj('region_true_list'+str(BATCH), region_true_list)
print('region_pred_list[:4]:', region_pred_list[:4])
print('region_true_list[:4]:', region_true_list[:4])
print('lista_erros[:8]:', lista_erros[:8])
print('len(lista_erros):', len(lista_erros))
print('len(set(lista_erros)):', len(set(lista_erros)))
print('len(region_true_list):', len(region_true_list))
print('len(region_pred_list):', len(region_pred_list))

print('-----Avaliando nested só com modelo de NER com BERT Segundo Nivel:-----')

print(classification_report(region_true_list, region_pred_list, digits=6))
print(confusion_matrix(region_true_list, region_pred_list))


Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
506
[[['Lucas', 0, 43], [',', 1, 48], ['74', 2, 50], ['anos', 3, 53], ['.', 4, 57]], []]
numero de sentencas no total: 506
idx2tag: {0: 'O', 1: 'Anatomia', 2: 'Problema', 3: 'Tratamento', 4: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 506
len(dicSentences_new_test): 506
len(dic_predictions): 506
region_pred_list[:4]: ['O', 'O', 'O', 'O']
region_true_list[:4]: ['Problema', 'Tratamento', 'Problema', 'Problema']
lista_erros[:8]: [1, 1, 2, 2, 2, 2, 2, 2]
len(lista_erros): 911
len(set(lista_erros)): 395
len(region_true_list): 1054
len(region_pred_list): 1054
-----Avaliando nested só com modelo de NER com BERT Segundo Nivel:-----
              precision    recall  f1-score   support

    Anatomia   0.650485  0.683673  0.666667       196
           O   0.000000  0.000000  0.000000        63
    Problema   0.000000  0.000000  0.000000       338
       Teste   0.000000  0.000000  0.000

C:\Users\lisat\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
region_true_list, region_pred_list, lista_erros = f.getListaRegionsTruePred(BATCH, dicSentences_new_test, r)
print(classification_report(region_true_list, region_pred_list, digits=6))
print(confusion_matrix(region_true_list, region_pred_list))

              precision    recall  f1-score   support

    Anatomia   0.669421  0.826531  0.739726       196
           O   0.000000  0.000000  0.000000       212
    Problema   0.798319  0.843195  0.820144       338
       Teste   0.878049  0.888889  0.883436       243
  Tratamento   0.800905  0.827103  0.813793       214

    accuracy                       0.698254      1203
   macro avg   0.629339  0.677144  0.651420      1203
weighted avg   0.653199  0.698254  0.674166      1203

[[162  28   4   1   1]
 [ 77   0  64  29  42]
 [  2  51 285   0   0]
 [  0  26   0 216   1]
 [  1  32   4   0 177]]
